# Workspace for Project Euler problems

## Problem 7

By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13.

What is the 10 001st prime number?

In [1]:
import primetools as pt

In [2]:
pt.findSomePrimes(20)

{2: 0, 3: 0, 5: 0, 7: 0, 11: 0, 13: 0, 17: 0, 19: 0}

In [3]:
pt.findSomePrimes(number=20)

{2: 0,
 3: 0,
 5: 0,
 7: 0,
 11: 0,
 13: 0,
 17: 0,
 19: 0,
 23: 0,
 29: 0,
 31: 0,
 37: 0,
 41: 0,
 43: 0,
 47: 0,
 53: 0,
 59: 0,
 61: 0,
 67: 0,
 71: 0}

In [10]:
def testFunc(a, b = None):
    print ("a is: {}".format(a))
    print ("b is: {}".format(b))
    

In [11]:
testFunc(1)

a is: 1
b is: None
